### Setup

#### Load "dotenv" extension to use enviroment variables in the Notebook

In [16]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


#### Load required libraries

In [17]:
from sqlalchemy import create_engine, text
import psycopg2 as ps
import pandas as pd
import os

#### Setup database connection

Make sure you create a `.env` file in the same directory as this notebook.

Your `.env` file should contain the definition of the following variables:
- DB_HOSTNAME
- DB_PORT
- DB_NAME
- DB_USER
- DB_PASS


Example:
```
DB_HOSTNAME = "localhost"
DB_PORT = 5432
DB_NAME = "retail_db"
DB_USER = "retail_user"
DB_PASS = "retailpassword"
```

In [18]:
DB_HOST = os.getenv("DB_HOSTNAME")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
    
DB_CONNECTION_URI = "postgresql://%s:%s@%s:%s/%s" % (DB_USER, DB_PASS, DB_HOST, DB_PORT, DB_NAME)

# Create SQL engine with SQLAlchemy
sql_engine = create_engine(DB_CONNECTION_URI)

> You can now execute queries to the database with pandas!

# Exercises - Basic SQL Queries

Here are some of the exercises for which you can write SQL queries to self evaluate using all the concepts we have learnt to write SQL Queries.

- All the exercises are based on the reatail database (retail tables).
- We have already setup the tables and also populated the data.
- We will use all the 6 tables in retail database as part of these exercises.

## Solutions

**NOTE:** You can run queries using pandas and SQLAlchemy as it follows:

```python
sql_df = pd.read_sql_query(
    '''
        SELECT *
        FROM courses
        ORDER BY course_id;
    ''',
    con = sql_engine
)
 
# Show query results
sql_df
```

### Exercise 1 - Customer order count

Get order count per customer for the month of 2014 January.

- Tables - `orders` and `customers`
- Data should be sorted in descending order by count and ascending order by customer id
- Output should contain `customer_id`, `customer_fname`, `customer_lname` and `customer_order_count`.

In [19]:
sql_df = pd.read_sql_query(
    '''
    SELECT customer_id, customer_fname, customer_lname, COUNT(*) AS customer_order_count
    FROM orders o
    LEFT JOIN customers c
    ON o.order_customer_id=c.customer_id
    WHERE order_date BETWEEN '2014-01-01' AND '2014-01-31'
    GROUP BY customer_id, customer_fname, customer_lname
    ORDER BY customer_order_count DESC, customer_id
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,customer_id,customer_fname,customer_lname,customer_order_count
0,8622,Shirley,Smith,5
1,9676,Theresa,Smith,5
2,7,Melissa,Wilcox,4
3,222,Frank,Ruiz,4
4,2444,Kenneth,Smith,4
...,...,...,...,...
4691,12424,Judy,Phillips,1
4692,12426,Jordan,Valdez,1
4693,12427,Mary,Smith,1
4694,12430,Hannah,Brown,1


### Exercise 2 - Dormant Customers

Get the customer details who have not placed any order for the month of 2014 january.

- Tables - `orders` and `customers`
- Output columns - All columns from `customers`
- Data should be sorted in ascending order by `customer_id`
- Output should contain all the fields from `customers`

> HINT: You can use `NOT IN` or `NOT EXISTS` or `OUTER JOIN` to solve this problem.

In [20]:
sql_df = pd.read_sql_query(
    '''
    SELECT DISTINCT *
    FROM orders o
    LEFT JOIN customers c
    ON o.order_customer_id=c.customer_id
    WHERE order_date NOT BETWEEN '2014-01-01' AND '2014-01-31'
    ORDER BY customer_id
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,order_id,order_date,order_customer_id,order_status,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode
0,22945,2013-12-13,1,COMPLETE,1,Richard,Hernandez,XXXXXXXXX,XXXXXXXXX,6303 Heather Plaza,Brownsville,TX,78521
1,15192,2013-10-29,2,PENDING_PAYMENT,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
2,33865,2014-02-18,2,COMPLETE,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
3,57963,2013-08-02,2,ON_HOLD,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
4,67863,2013-11-30,2,COMPLETE,2,Mary,Barrett,XXXXXXXXX,XXXXXXXXX,9526 Noble Embers Ridge,Littleton,CO,80126
...,...,...,...,...,...,...,...,...,...,...,...,...,...
62970,42915,2014-04-16,12434,COMPLETE,12434,Mary,Mills,XXXXXXXXX,XXXXXXXXX,9720 Colonial Parade,Caguas,PR,00725
62971,51800,2014-06-14,12434,ON_HOLD,12434,Mary,Mills,XXXXXXXXX,XXXXXXXXX,9720 Colonial Parade,Caguas,PR,00725
62972,61777,2013-12-26,12434,COMPLETE,12434,Mary,Mills,XXXXXXXXX,XXXXXXXXX,9720 Colonial Parade,Caguas,PR,00725
62973,41643,2014-04-08,12435,PENDING,12435,Laura,Horton,XXXXXXXXX,XXXXXXXXX,5736 Honey Downs,Summerville,SC,29483


### Exercise 3

Get the revenue generated by each customer for the month of 2014 january.

- Tables - `orders`, `order_items` and `customers`
- Data should be sorted in descending order by revenue and then ascending order by `customer_id`
- Output should contain `customer_id`, `customer_fname`, `customer_lname`and `customer_revenue`.
- If there are no orders placed by customer, then the corresponding revenue for a given customer should be 0.
- Consider only `COMPLETE` and `CLOSED` orders.

In [21]:
sql_df = pd.read_sql_query(
    '''
    SELECT customer_id,customer_fname,customer_lname, SUM(order_item_subtotal) AS customer_revenue
    FROM order_items oi
    LEFT JOIN orders o
    ON oi.order_item_order_id=o.order_id
    LEFT JOIN customers c
    ON o.order_customer_id=c.customer_id
    WHERE order_date BETWEEN '2014-01-01' AND '2014-01-31' AND order_status IN ('COMPLETE','CLOSED')
    GROUP BY customer_id,customer_fname,customer_lname
    ORDER BY customer_revenue DESC, customer_id
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,customer_id,customer_fname,customer_lname,customer_revenue
0,2555,Mary,Long,2954.63
1,3465,Mary,Gardner,2929.74
2,3710,Ashley,Smith,2739.82
3,1780,Larry,Sharp,2689.65
4,986,Catherine,Hawkins,2629.90
...,...,...,...,...
1936,738,Mary,Wood,24.99
1937,212,Roger,Johnson,19.99
1938,2359,Emily,Hubbard,19.98
1939,3554,Michael,Smith,14.99


### Exercise 4

Get the revenue generated for each category for the month of 2014 january.


- Tables - `orders`, `order_items`, `products` and `categories`
- Data should be sorted in ascending order by `category_id`
- Output should contain all the field from `categories` along with the revenue as `category_revenue`
- Consider only `COMPLETE` and `CLOSED` orders.

In [22]:
sql_df = pd.read_sql_query(
    '''
    SELECT category_id, category_department_id, category_name, SUM(order_item_subtotal) AS category_revenue 
    FROM order_items oi
    LEFT JOIN orders o
    ON oi.order_item_order_id = o.order_id
    LEFT JOIN products p
    ON oi.order_item_product_id = p.product_id
    LEFT JOIN categories c
    ON c.category_id = p.product_category_id
    WHERE order_date BETWEEN '2014-01-01' AND '2014-01-31' AND order_status IN ('COMPLETE', 'CLOSED')
    GROUP BY category_id, category_department_id, category_name
    ORDER BY category_id
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,category_id,category_department_id,category_name,category_revenue
0,2,2,Soccer,1094.88
1,3,2,Baseball & Softball,3214.41
2,4,2,Basketball,1299.98
3,5,2,Lacrosse,1299.69
4,6,2,Tennis & Racquet,1124.75
5,7,2,Hockey,1433.00
6,9,3,Cardio Equipment,133156.77
7,10,3,Strength Training,3388.96
8,11,3,Fitness Accessories,1509.73
9,12,3,Boxing & MMA,3998.46


### Exercise 5

Get the count of products for each department.
- Tables - `departments`, `categories`, `products``
- Data should be sorted in ascending order by `deparment_id``
- Output should contain all the fields from departments and the product count as `product_count`

In [23]:
sql_df = pd.read_sql_query(
    '''
    SELECT department_id, department_name, COUNT(DISTINCT product_id) AS product_count 
    FROM products p
    LEFT JOIN categories c
    ON p.product_category_id = c.category_id
    LEFT JOIN departments d
    ON d.department_id = c.category_department_id
    GROUP BY department_id, department_name
    ''',
    con = sql_engine
)
 
# Show query results
sql_df

,department_id,department_name,product_count
0,2.0,Fitness,168
1,3.0,Footwear,168
2,4.0,Apparel,140
3,5.0,Golf,120
4,6.0,Outdoors,336
5,7.0,Fan Shop,149
6,NaN,None,264
